In [8]:
import yfinance as yf
import pandas as pd

tickers = ['1454.TW', '2539.TW', '4438.TW', '3617.TW', '6139.TW']

start_date = '2021-01-01'
end_date = '2023-12-31'

data = {}
for ticker in tickers:
    data[ticker] = yf.download(ticker, start=start_date, end=end_date)

# 將資料合併到一個 DataFrame 中
df = pd.concat(data.values(), axis=1)

# 將資料存儲到 CSV 檔案中
df.to_csv('stock_data.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
df2=yf.download('IR0001'+'.TW',start='2021-01-01', period='max', interval='1d')
df2.to_csv('yahoo'+'IR0001'+'TW.csv')

[*********************100%%**********************]  1 of 1 completed


In [19]:
tickers = ['1454', '2539', '4438', '3617', '6139']
tickers.append('IR0001.TW')

In [20]:
print(tickers)

['1454', '2539', '4438', '3617', '6139', 'IR0001.TW']


In [37]:
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
def make_pipeline():
    # 計算短期移動平均線（Short-term Moving Average）
    short_ma = SimpleMovingAverage(inputs=[EquityPricing.close], window_length=50)
    
    # 計算長期移動平均線（Long-term Moving Average）
    long_ma = SimpleMovingAverage(inputs=[EquityPricing.close], window_length=200)
    
    # 計算當前價格
    curr_price = EquityPricing.close.latest
    
    # 根據道氏理論，判斷趨勢
    trend = (short_ma > long_ma) & (curr_price > short_ma)
    
    # 計算相對強弱指數（RSI）
    delta = EquityPricing.close.diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.rolling(window=14).mean()
    roll_down = down.rolling(window=14).mean().abs()
    rs = roll_up / roll_down
    rsi = 100 - (100 / (1 + rs))
    
    return Pipeline(
        columns = {
            'hort_ma': short_ma,
            'long_ma': long_ma,
            'curr_price': curr_price,
            'trend': trend,
            'rsi': rsi
        }
    )

In [33]:
def initialize(context):
    context.last_buy_price = 0
    set_slippage(slippage.VolumeShareSlippage())
    set_commission(commission.PerShare(cost=0.00285))
    set_benchmark(symbol('IR0001'))
    attach_pipeline(make_pipeline(), 'mystrategy')
    context.last_signal_price = 0

In [34]:
#如果短期移動平均線高於長期移動平均線，且當前價格高於短期移動平均線，且 RSI 小於 30，則買入股票。
#如果短期移動平均線低於長期移動平均線，或者當前價格低於短期移動平均線，或者 RSI 高於 70，則賣出股票。
#假設了股票的交易量為 1000 股，並且使用 context.last_signal_price 變數來記錄最後一次買賣的價格。
def handle_data(context, data):
    out_dir = pipeline_output('mystrategy')
    for i in out_dir.index: 
        short_ma = out_dir.loc[i, 'hort_ma']
        long_ma = out_dir.loc[i, 'long_ma']
        curr_price = out_dir.loc[i, 'curr_price']
        rsi = out_dir.loc[i, 'rsi']
        cash_position = context.portfolio.cash
        stock_position = context.portfolio.positions[i].amount
        
        buy, sell = False, False
        record(price = curr_price, short_ma = short_ma, long_ma = long_ma, rsi = rsi, buy = buy, sell = sell)
        
        if stock_position == 0:
            if (short_ma > long_ma) and (curr_price > short_ma) and (rsi < 30) and (cash_position >= curr_price * 1000):
                order(i, 1000)
                context.last_signal_price = curr_price
                buy = True
                record(buy = buy)
        elif stock_position > 0:
            if (short_ma < long_ma) or (curr_price < short_ma) or (rsi > 70):
                order_target(i, 0)
                context.last_signal_price = 0
                sell = True
                record(sell = sell)
            else:
                pass
        else:
            pass

In [35]:
def analyze(context, perf):
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel("Portfolio value (NTD)")
    ax2 = fig.add_subplot(212)
    ax2.set_ylabel("Price (NTD)")
    perf.price.plot(ax=ax2)
    perf.short_ma.plot(ax=ax2)
    perf.long_ma.plot(ax=ax2)
    ax2.plot( # 繪製買入訊號
        perf.index[perf.buy],
        perf.loc[perf.buy, 'price'],
        '^',
        markersize=5,
        color='red'
    )
    ax2.plot( # 繪製賣出訊號
        perf.index[perf.sell],
        perf.loc[perf.sell, 'price'],
        'v',
        markersize=5,
        color='green'
    )
    plt.legend(loc=0)
    plt.gcf().set_size_inches(18,8)
    plt.show()